In [5]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split

path='data/CLUSTERED'
data_path='data'
ref_dataset= np.load(os.path.join(path,'refrigerator.dat.npy'))
print(ref_dataset[0])
ref_bins=4
ref_window_size=2401
input_shape=(2401, 1)

bin_max_count=[0 for i in range(ref_bins)]

[222.2   0. ]


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Dropout,Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import BatchNormalization

def load_cnn():
    
    model=Sequential()

    model.add(Conv1D(filters=8,kernel_size=5,input_shape=input_shape))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=8,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=16,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=32,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    model.add(Conv1D(filters=64,kernel_size=3))
    model.add(BatchNormalization())
    #model.add(Activation('relu'))
    model.add(MaxPooling1D(pool_size=4))    
    model.add(Flatten())
    model.add(Dense(100,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3,activation='sigmoid'))

    from keras.optimizers import Adam
    
    learning_rate = 0.1
    decay_rate = 2
    momentum = 0.8
    opt= Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=decay_rate)
    
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    
    return model

In [9]:
import collections
from keras.utils import np_utils

bin_dict=collections.Counter(ref_dataset[:,1])

ref_data=[]
ref_target=[]

model=load_cnn()
j=0
key_min = min(bin_dict.keys(), key=(lambda k: bin_dict[k]))
ref_min_bin=bin_dict[key_min]
print(ref_min_bin)
for i in range(ref_dataset.shape[0]-ref_window_size-1):
    ref_data.append(ref_dataset[i:i+ref_window_size,0])
    ref_target.append(ref_dataset[i+ref_window_size-1,1])
    j=j+1
    if(j==20000):
        data=np.array(ref_data)
        target=np.array(ref_target)
        data=data.reshape(data.shape[0],data.shape[1],1)

        target = np_utils.to_categorical(target)

        train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

        train_data=train_data.reshape(train_data.shape[0],train_data.shape[1],1)
        test_data=test_data.reshape(test_data.shape[0],test_data.shape[1],1)

        history=model.fit(train_data,train_target,epochs=2) 
        print(model.evaluate(test_data,test_target))
        ref_data=[]
        ref_target=[]
        j=0

11334
Epoch 1/2
18000/18000 [==============================] - 56s 3ms/step - loss: 0.2157 - acc: 0.9097
Epoch 2/2
2000/2000 [==============================] - 3s 1ms/step
[nan, 0.63]
Epoch 1/2
18000/18000 [==============================] - 53s 3ms/step - loss: nan - acc: 0.7764
Epoch 2/2
2000/2000 [==============================] - 2s 1ms/step
[nan, 0.7835]


ValueError: Error when checking target: expected dense_8 to have shape (3,) but got array with shape (2,)